In [1]:
import pandas as pd
import re

In [ ]:
def split_text(text: str) -> list:
    text = text.replace('\n\n', '<p>')
    text = ' '.join(text.split())
    text = text.replace('<p>', '\n')
    # text = text.lower()
    return text

In [2]:
df_path = '../data/parsed/kiids_parsed.parquet'

In [3]:
df = pd.read_parquet(df_path)

In [4]:
df_practial_info = df['informacje praktyczne']
df_practial_info = df_practial_info[~df_practial_info.isna()]

In [5]:
df_practial_info

0      INFORMACJE PRAKTYCZNE\nNiniejszy dokument opis...
1      Informacje praktyczne \nDepozytariusz: Bank Pe...
2                               INFORMACJE PRAKTYCZNE \n
3      Informacje praktyczne \nSubfundusz został utwo...
5      Informacje praktyczne \nDepozytariusz: Bank Pe...
                             ...                        
396    Informacje praktyczne \nSubfundusz został utwo...
397    Informacje praktyczne \nDepozytariusz: Bank Pe...
398                             INFORMACJE PRAKTYCZNE \n
399    Informacje praktyczne \nKluczowe informacje dl...
400    Informacje praktyczne\nKluczowe informacje dla...
Name: informacje praktyczne, Length: 391, dtype: object

In [7]:
df_practial_info = df_practial_info.apply(split_text)

In [8]:
df_practial_info

0      INFORMACJE PRAKTYCZNE Niniejszy dokument opisu...
1      Informacje praktyczne Depozytariusz: Bank Peka...
2                                  INFORMACJE PRAKTYCZNE
3      Informacje praktyczne Subfundusz został utworz...
5      Informacje praktyczne Depozytariusz: Bank Peka...
                             ...                        
396    Informacje praktyczne Subfundusz został utworz...
397    Informacje praktyczne Depozytariusz: Bank Peka...
398                                INFORMACJE PRAKTYCZNE
399    Informacje praktyczne Kluczowe informacje dla ...
400    Informacje praktyczne Kluczowe informacje dla ...
Name: informacje praktyczne, Length: 391, dtype: object

# Depositary

In [47]:
depo_regex = re.compile('depozytariusz(em.*jest)?:?\s*(.*?)(S\.A)?(\.|,|;)', re.IGNORECASE)
not_depo_regex = re.compile('(niezgodne.*)|(to\s)')

In [48]:
def get_depo(text: str) -> str or None:
    regex = depo_regex.search(text)
    if regex is None:
        return None
    else:     
        if not_depo_regex.search(regex.group(2)):
            return None
        
        if regex.group(3) is not None:
            return regex.group(2) + regex.group(3)
        else:
            return regex.group(2)

In [49]:
df_depo_re = df_practial_info.apply(get_depo)

In [50]:
df_depo_re[~df_depo_re.isna()].unique()

array(['Bank Handlowy w Warszawie SA', 'Bank Pekao S.A',
       'Depozytariuszem funduszu jest mBank S.A',
       'Deutsche Bank Polska Akcyjna z siedzibą w Warszawie',
       'ING Bank Śląski S.A', 'PKO Bank Polski S.A',
       'Deutsche Bank Polska Spółka Akcyjna z siedzibą w Warszawie',
       'mBank S.A', 'BNP Paribas Bank Polska S.A',
       'określona w tych umowach lub zasadach',
       'Bank Handlowy w Warszawie S.A',
       'Bank Polska Kasa Opieki Spółka Akcyjna z siedzibą w Warszawie'],
      dtype=object)


# KRS

In [76]:
krs_regex = re.compile('(krs:?\s*)([0-9]{9})', re.IGNORECASE)

In [86]:
def get_krs(text: str) -> str or None:
    regex = krs_regex.search(text)
    if regex is None:
        return None
    else:
        return regex.group(2)

In [87]:
krs = df_practial_info.apply(get_krs)

In [88]:
krs[~krs.isna()]

9      000003112
134    000003112
300    000003112
Name: informacje praktyczne, dtype: object

# NIP

In [131]:
nip_regex = re.compile('(nip:?\s*)([0-9\-]*)(\s|\.|,)', re.IGNORECASE)

In [135]:
def get_nip(text: str) -> str:
    regex = nip_regex.search(text)
    if regex is None:
        return None
    else:
        return regex.group(2)

In [136]:
nip = df_practial_info.apply(get_nip)

In [137]:
nip[~nip.isna()]

9      526-02-10-808
134    526-02-10-808
300    526-02-10-808
Name: informacje praktyczne, dtype: object

# SZIEDZIBA_TOWARZYSTWA

In [228]:
address_regex = re.compile('(ul\.\s\w+\s\d{1,3},?\s\d{2}-\d{3}\s\w+)')

In [232]:
def get_address(text: str) -> str:
    regex = address_regex.search(text)
    if regex is None:
        return None
    else:
        return regex.group(1)

In [233]:
addressess = df_practial_info.apply(get_address)

In [234]:
addressess[~addressess.isna()]

9       ul. Grzybowskiej 78, 00-844 w
28     ul. Prosta 18, 00-950 Warszawa
68     ul. Prosta 18, 00-950 Warszawa
83     ul. Prosta 18, 00-950 Warszawa
121    ul. Prosta 18, 00-950 Warszawa
124    ul. Prosta 18, 00-950 Warszawa
131    ul. Prosta 18, 00-950 Warszawa
133    ul. Prosta 18, 00-950 Warszawa
134     ul. Grzybowskiej 78, 00-844 w
137    ul. Prosta 18, 00-950 Warszawa
141    ul. Prosta 18, 00-950 Warszawa
150    ul. Prosta 18, 00-950 Warszawa
231    ul. Prosta 18, 00-950 Warszawa
300     ul. Grzybowskiej 78, 00-844 w
327    ul. Prosta 18, 00-950 Warszawa
358    ul. Prosta 18, 00-950 Warszawa
374    ul. Prosta 18, 00-950 Warszawa
384    ul. Prosta 18, 00-950 Warszawa
399    ul. Prosta 18, 00-950 Warszawa
Name: informacje praktyczne, dtype: object

# KAPITAL_ZAKLADOWY_TOWARZYSTWA

In [264]:
capital_regex = re.compile('zakładowy:?.{,20}([\d\s,]+)', re.IGNORECASE)

In [265]:
def get_capital(text: str) -> str:
    regex = capital_regex.search(text)
    
    if regex is None:
        return None
    else:
        return regex.group(1)

In [266]:
capital = df_practial_info.apply(get_capital)

In [267]:
capital[~capital.isna()]

9       612,00 
134     612,00 
300     612,00 
Name: informacje praktyczne, dtype: object

# WALUTA_KAPITALU_ZAKLADOWEGO_TO

In [298]:
currency_regex = re.compile('(przedstawiono\sw\s)(\w{3})', re.IGNORECASE)

In [299]:
def get_currency(text: str) -> str:
    regex = currency_regex.search(text)
    if regex is None:
        return None
    else:
        return regex.group(2)   

In [300]:
currency = df_practial_info.apply(get_currency)

In [304]:
# currency[~currency.isna()]

# CZY_ESG

In [333]:
esg_regex = re.compile('esg' , re.IGNORECASE)
not_esg_regex = re.compile('.{0,10}nie.{0,10}esg.{0,10}nie.{0,10}')

In [329]:
def get_esg(text: str) -> str:
    regex = esg_regex.search(text)
    
    if regex is None:
        return False
    else:
        not_regex = not_esg_regex.search(text)
        if not_regex is None:
            return False
        return True

In [330]:
esg = df_practial_info.apply(get_esg)

In [331]:
esg[~esg.isna()]

11     Informacje praktyczne Amundi Stars Global Ecol...
60     Informacje praktyczne Amundi Stars Global Ecol...
228    westor ma prawo dokonywać zamiany jednostek uc...
241    y. Inwestor ma prawo dokonywać zamiany jednost...
265    Informacje praktyczne Amundi Stars Global Ecol...
Name: informacje praktyczne, dtype: object

In [332]:
esg.iloc[10]

'Informacje praktyczne Amundi Stars Global Ecology ESG Informacje praktyczne Niniejsze Kluczowe informacje dla inwestorów opisują Amundi Stars Global Ecol'

# TYP_FUNDUSZU
wyjątkowo `intro`

In [343]:
raw_text = df['raw_text']
raw_text = intro[~raw_text.isna()]

In [344]:
raw_text = raw_text.apply(split_text)

In [345]:
type_regex = re.compile('(S?FIO)')

In [346]:
def get_type(text: str) -> str:
    regex = type_regex.search(text)
    
    if regex is None:
        return None
    else:
        return regex.group(1)

In [347]:
types = raw_text.apply(get_type)

In [351]:
types[~types.isna()]

10     SFIO
22     SFIO
33     SFIO
34     SFIO
39     SFIO
       ... 
380    SFIO
382    SFIO
388    SFIO
394    SFIO
397    SFIO
Name: intro, Length: 64, dtype: object